In [1]:
import os
import sys
import importlib
from pathlib import Path

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

# %load_ext autoreload
# %autoreload 2

In [427]:
# MAIN DIR
main_dir = f'{str(Path(code_dir).parents[0])}/'

# code_dir
code_dir = f'{code_dir}/'
sys.path.append(code_dir)

# scraping dir
scraped_data = f'{code_dir}scraped_data/'

# data dir
data_dir = f'{code_dir}data/'

# lang models dir
llm_path = f'{data_dir}Language Models'

# sites
site_list=['Indeed', 'Glassdoor', 'LinkedIn']

# columns
cols=['Sector', 
      'Sector Code', 
      'Gender', 
      'Age', 
      'Language', 
      'Dutch Requirement', 
      'English Requirement', 
      'Gender_Female', 
      'Gender_Mixed', 
      'Gender_Male', 
      'Age_Older', 
      'Age_Mixed', 
      'Age_Younger', 
      'Gender_Num', 
      'Age_Num', 
      '% Female', 
      '% Male', 
      '% Older', 
      '% Younger']

int_variable: str = 'Job ID'
str_variable: str = 'Job Description'
gender: str = 'Gender'
age: str = 'Age'
language: str = 'en'
str_cols = ['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name', 'Location', 'Job Description', 'Company URL', 'Job URL', 'Tracking ID']
pattern = r'[\n]+|[,]{2,}|[|]{2,}|[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?=\:+[A-Z])'
pattern_numbers = r'[^a-zA-Z0-9]'

In [3]:
import string
import re
import time
import json
import csv
import glob
import pickle
import pandas as pd
import numpy as np
import pygwalker as pyg
import googletrans
from googletrans import Translator
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tag import pos_tag, pos_tag_sents
from spacy.pipeline import Sentencizer


In [383]:
df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw_dropped.pkl').reset_index(drop=True)


In [384]:
len(df_jobs)


62577

In [385]:
df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].apply(
    lambda job_description: ' '.join(job_description.split('/')) if '/' in job_description else job_description
)


In [386]:
abb_dict = {
    r'incl\.': 'including', 
    r'e\.g\.': 'for example', 
    r'e\.g': 'for example', 
    r'etc\.': 'et cetera', 
}


In [387]:
str_fix_incl = 'Apply appropriate and effective communication methods to senior management and important stakeholders'
str_fix_eg = 'Partner with Procurement in order to manage suppliers for the projects & programs in scope'



In [388]:
df_jobs['Job Description'] = df_jobs['Job Description'].replace(abb_dict, regex=True)

In [389]:
df_jobs['Job Description'][df_jobs['Job Description'].str.contains(str_fix_incl)]


35      Purpose\n\nAs a Studio A Director\n\nSpecial P...
3423    Purpose:\nAs a Manager Digital Strategy and Pr...
Name: Job Description, dtype: object

In [390]:
re.findall(str_fix_eg, df_jobs['Job Description'][35])

['Partner with Procurement in order to manage suppliers for the projects & programs in scope']

In [391]:
list(re.finditer(str_fix_eg, df_jobs['Job Description'][35]))

[<re.Match object; span=(1461, 1551), match='Partner with Procurement in order to manage suppl>]

In [392]:
df_jobs['Job Description'][35][1461:1580]

'Partner with Procurement in order to manage suppliers for the projects & programs in scope, for example interviews, sta'

In [393]:
# df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].apply(
#     lambda job_description: job_description.replace('incl.', 'including')
# )

# df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].apply(
#     lambda job_description: job_description.replace('e.g.', 'e.g')
# )

# df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].apply(
#     lambda job_description: job_description.replace('e.g', 'for example')
# )

# df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].apply(
#     lambda job_description: job_description.replace('etc.', 'et cetera')
# )


In [394]:
# df_jobs = df_jobs[:100]

In [395]:
# df_jobs = df_jobs.sample(n=100, random_state=1)

In [396]:
# df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw_glob_paths_10.pkl').reset_index(drop=True)


In [397]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62577 entries, 0 to 62576
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Search Keyword     62577 non-null  object 
 1   Platform           62577 non-null  object 
 2   Job ID             62577 non-null  object 
 3   Job Title          62577 non-null  object 
 4   Company Name       62574 non-null  object 
 5   Location           62577 non-null  object 
 6   Job Description    62577 non-null  object 
 7   Rating             3975 non-null   float64
 8   Employment Type    61995 non-null  object 
 9   Company URL        59263 non-null  object 
 10  Job URL            62577 non-null  object 
 11  Job Age            62577 non-null  object 
 12  Job Age Number     62577 non-null  object 
 13  Collection Date    62577 non-null  object 
 14  Data Row           58599 non-null  float64
 15  Tracking ID        58599 non-null  object 
 16  Industry           591

In [398]:
df_jobs.columns


Index(['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name',
       'Location', 'Job Description', 'Rating', 'Employment Type',
       'Company URL', 'Job URL', 'Job Age', 'Job Age Number',
       'Collection Date', 'Data Row', 'Tracking ID', 'Industry', 'Job Date',
       'Type of ownership'],
      dtype='object')

In [637]:
df_sectors = pd.read_pickle(f'{scraped_data}CBS/Data/Sectors Output from script.pkl').reset_index(drop=True)


In [646]:
df_sectors

,Code,Sector Name,Keywords,Keywords Count,% per Sector,% per Social Category,% per Workforce,Gender_Female_n,Gender_Female_% per Sector,Gender_Female_% per Social Category,...,Age_Older (>= 45 years)_% per Sector,Age_Older (>= 45 years)_% per Social Category,Age_Older (>= 45 years)_% per Workforce,Age_Younger (< 45 years)_n,Age_Younger (< 45 years)_% per Sector,Age_Younger (< 45 years)_% per Social Category,Age_Younger (< 45 years)_% per Workforce,Age_Sectoral Age Segregation_Dominant Category,n,% Sector per Workforce
0,A-U,All economic activities,NaN,NaN,NaN,NaN,NaN,4029.0,NaN,NaN,...,NaN,NaN,NaN,4892.0,NaN,NaN,NaN,NaN,8391.0,NaN
1,A,Agriculture and industry,"[forestry, agriculture, fishing]",3.0,0.002144,0.030303,0.000119,310.0,0.221587,0.025797,...,0.493209,0.064905,0.027305,708.0,0.506076,0.048394,0.028017,Mixed Age,1399.0,0.055362
2,B,"Industry (no construction), energy","[mining, quarry, quarrying]",3.0,0.001484,0.030303,0.000119,423.0,0.209199,0.035200,...,0.527201,0.100273,0.042184,954.0,0.471810,0.065208,0.037752,Older,2022.0,0.080016
3,C,Manufacturing,"[gas supply, industry, managing director, prod...",7.0,0.009138,0.070707,0.000277,174.0,0.227154,0.014479,...,0.539164,0.038849,0.016343,354.0,0.462141,0.024197,0.014009,Older,766.0,0.030313
4,D,Energy supply,"[energy, energy supply, electricity]",3.0,0.103448,0.030303,0.000119,8.0,0.275862,0.000666,...,0.517241,0.001411,0.000594,13.0,0.448276,0.000889,0.000514,Mixed Age,29.0,0.001148
5,E,Water supply and waste management,"[waste management, water, water stock, water s...",4.0,0.111111,0.040404,0.000158,7.0,0.194444,0.000583,...,0.583333,0.001975,0.000831,16.0,0.444444,0.001094,0.000633,Older,36.0,0.001425
6,F,Construction,"[construction, build]",2.0,0.005952,0.020202,0.000079,42.0,0.125000,0.003495,...,0.476190,0.015050,0.006332,178.0,0.529762,0.012167,0.007044,Mixed Age,336.0,0.013296
7,G,Commercial services,"[retail, selling, retail trade, wholesale, man...",11.0,0.001387,0.111111,0.000435,3421.0,0.431345,0.284680,...,0.340941,0.254350,0.107004,5228.0,0.659185,0.357348,0.206886,Mixed Age,7931.0,0.313850
8,H,Transportation and storage,"[transportation, logistics, transport, storage...",5.0,0.012953,0.050505,0.000198,95.0,0.246114,0.007905,...,0.531088,0.019283,0.008112,181.0,0.468912,0.012372,0.007163,Older,386.0,0.015275
9,I,Accommodation and food serving,"[food, hotel, serving, service, accommodation,...",7.0,0.017677,0.070707,0.000277,199.0,0.502525,0.016560,...,0.189394,0.007055,0.002968,320.0,0.808081,0.021873,0.012663,Younger,396.0,0.015671


In [645]:
df_sectors.columns = [
    '_'.join(col) 
    if 'SBI Sector Titles' not in col 
    and 'Total Workforce' not in col 
    else col[-1] 
    for col in df_sectors.columns
]

In [649]:
type(df_sectors['Keywords'][1])

list

In [650]:
df_sectors = df_sectors.explode(
    'Keywords', ignore_index=True
).reset_index(drop=True)

In [652]:
pd.merge(
    df_jobs, df_sectors, left_on='Search Keyword', right_on='Keywords', right_index=True, how='left', sort=False
)

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [580]:
df_jobs.set_index('Search Keyword', inplace=True)
df_sectors.set_index('Keywords', inplace=True)
df3 = df_sectors.join(df_jobs, how='left')


/var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_35937/3937353499.py:3: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (3 levels on the left, 1 on the right)
  df3 = df_sectors.join(df_jobs, how='left')


In [581]:
df3

,"(SBI Sector Titles, Industry class / branch (SIC2008), Code)","(SBI Sector Titles, Industry class / branch (SIC2008), Sector Name)","(SBI Sector Titles, Industry class / branch (SIC2008), Keywords Count)","(SBI Sector Titles, Industry class / branch (SIC2008), % per Sector)","(SBI Sector Titles, Industry class / branch (SIC2008), % per Social Category)","(SBI Sector Titles, Industry class / branch (SIC2008), % per Workforce)","(Gender, Female, n)","(Gender, Female, % per Sector)","(Gender, Female, % per Social Category)","(Gender, Female, % per Workforce)",...,Job Description nlp_token_tags,Job Description nlp_lemmas,Job Description nlp_stems,num_words,num_unique_words,num_chars,num_punctuations,sentiment,1grams_bert,bert_tokenized
accommodation,I,Accommodation and food serving,7.0,0.017677,0.070707,0.000277,199.0,0.502525,0.016560,0.007875,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
agriculture,A,Agriculture and industry,3.0,0.002144,0.030303,0.000119,310.0,0.221587,0.025797,0.012268,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
application developer,J,Information and communication,9.0,0.031034,0.090909,0.000356,80.0,0.275862,0.006657,0.003166,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
author,J,Information and communication,9.0,0.031034,0.090909,0.000356,80.0,0.275862,0.006657,0.003166,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
build,F,Construction,2.0,0.005952,0.020202,0.000079,42.0,0.125000,0.003495,0.001662,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wholesale,G,Commercial services,11.0,0.001387,0.111111,0.000435,3421.0,0.431345,0.284680,0.135378,...,"[(ability, NN), (to, TO), (travel, VB), (,, ,)...","[ability, travel, domestic, international, req...","[abil, travel, domest, intern, requir]",8.0,8.0,57.0,2.0,0.3182,"[ability, to, travel, ,, domestic, or, interna...","[ability, to, travel, ,, domestic, or, interna..."
wholesale,G,Commercial services,11.0,0.001387,0.111111,0.000435,3421.0,0.431345,0.284680,0.135378,...,"[(in, IN), (-, HYPH), (depth, NN), (understand...","[depth, understanding]","[depth, understand]",4.0,4.0,28.0,1.0,0.0000,"[in, -, depth, understanding, of, it]","[in, -, depth, understanding, of, it]"
wholesale,G,Commercial services,11.0,0.001387,0.111111,0.000435,3421.0,0.431345,0.284680,0.135378,...,"[(fluent, NNP), (english, NNP), (both, CC), (v...","[fluent, english, verbally, write]","[fluent, english, verbal, written]",6.0,6.0,40.0,0.0,0.0000,"[fluent, english, both, verbal, ##ly, and, wri...","[fluent, english, both, verbal, ##ly, and, wri..."
NaN,A-U,All economic activities,NaN,NaN,NaN,NaN,4029.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [399]:
len(df_jobs.loc[(df_jobs['Job Description'].str.contains(str_fix_incl)) | (df_jobs['Job Description'].str.contains(str_fix_eg))])


2

In [400]:
# job_descriptions = list(set(df_jobs['Job Description'].to_list()))


In [401]:
job_descriptions = list(
    set(
        df_jobs['Job Description'].loc[
            (df_jobs['Job Description'].str.contains(str_fix_incl)) |
            (df_jobs['Job Description'].str.contains(str_fix_eg))
        ].to_list()
    )
)


In [402]:
len(job_descriptions)


2

In [403]:
job_descriptions[0][1500:2000]

'nage suppliers for the projects & programs in scope, for example interviews, staffing, supplier costs.\n\nCommunication   stakeholder management\nAct as the key contact person for strategic stakeholders.\nApply appropriate and effective communication methods to senior management and important stakeholders (including vendor) throughout the project lifecycle.\nAs conflicts and escalations arise within projects, identify solutions and manage the resolution in a timely and appropriate manner.\nDrive chang'

In [404]:
df_jobs = df_jobs.loc[
    (df_jobs['Job Description'].str.contains(str_fix_incl)) | 
    (df_jobs['Job Description'].str.contains(str_fix_eg))
].reset_index(drop=True)

In [405]:
len(df_jobs)

2

In [406]:
df_jobs.columns

Index(['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name',
       'Location', 'Job Description', 'Rating', 'Employment Type',
       'Company URL', 'Job URL', 'Job Age', 'Job Age Number',
       'Collection Date', 'Data Row', 'Tracking ID', 'Industry', 'Job Date',
       'Type of ownership'],
      dtype='object')

In [407]:
# Function to make a list of punctuations that determine sentence boundry, i.e., split characters
def make_custom_punct_chars(main_punct_chars = [':', '|'], repeated_punct_chars = ['\n', ',']):
    custom_punct_chars = []
    temp_multi = []
    temp_spaced = []

    for punct_char in main_punct_chars:
        custom_punct_chars+= f'{punct_char}', f'{punct_char} '

    for idx in range(4):
        for punct_char in repeated_punct_chars:
            temp_multi.append(f'{punct_char}'*int(idx+1))
            temp_spaced.append(f'{punct_char} '*int(idx+1))

    for multi, spaced in zip(temp_multi, temp_spaced):
        custom_punct_chars+= multi, spaced

    custom_punct_chars.remove(',')
    custom_punct_chars.remove(', ')

    return custom_punct_chars


In [408]:
custom_punct_chars = make_custom_punct_chars()

In [409]:
custom_punct_chars

[':',
 ': ',
 '|',
 '| ',
 '\n',
 '\n ',
 '\n\n',
 '\n \n ',
 ',,',
 ', , ',
 '\n\n\n',
 '\n \n \n ',
 ',,,',
 ', , , ',
 '\n\n\n\n',
 '\n \n \n \n ',
 ',,,,',
 ', , , , ']

In [410]:
# Set up Spacy
import spacy
from spacy.symbols import NORM, ORTH, LEMMA, POS

nlp = spacy.load('en_core_web_sm')


In [411]:
sentencizer = nlp.add_pipe('sentencizer')
sentencizer.punct_chars.update(custom_punct_chars)


In [412]:
if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    with open(f'{data_dir}punctuations.txt', 'wb') as f:
        pickle.dump(sentencizer.punct_chars, f)

In [413]:
with open(f'{data_dir}punctuations.txt', 'rb') as f:
    custom_punct_char = pickle.load(f)

In [414]:
from spacy.symbols import NORM, ORTH, LEMMA

special_cases_dict = {
    'incl.': [{65: 'incl', 67: 'including'}],
    'incl. ': [{65: 'incl', 67: 'including'}],
    '(incl.': [{65: 'incl', 67: 'including'}],
    'etc.': [{65: 'etc', 67: 'et cetera'}],
    'etc. ': [{65: 'etc', 67: 'et cetera'}],
    'e.g.': [{65: 'e.g', 67: 'for example'}],
    'e.g. ': [{65: 'e.g', 67: 'for example'}],
}

nlp.tokenizer.rules.update(special_cases_dict)


In [415]:
nlp.tokenizer.rules['e.g.']

[{65: 'e.g', 67: 'for example'}]

In [420]:
if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    df_jobs['Job Description spacy_sentencized'] = df_jobs['Job Description'].apply(
        lambda job_description: [
            sent 
            for sentence in nlp(job_description).sents 
            for sent in re.split(pattern, sentence.text) 
            if len(sent) != 0 
        ]
    )


In [423]:
df_jobs['Job Description spacy_sentencized']

0                                                Purpose
1                                 As a Studio A Director
2                                       Special Projects
3      in adidas Digital your task is to manage and d...
4      The program scope may comprise multiple countr...
                             ...                        
130    Strong MS-Office skills (Word, Excel, PowerPoint)
131     Basic Experience and a broad understanding of IT
132    Working knowledge of Agile working methods for...
133    Working knowledge of PMI methods ideally with ...
134             Fluent English both verbally and written
Name: Job Description spacy_sentencized, Length: 135, dtype: object

In [422]:
df_jobs = df_jobs.explode('Job Description spacy_sentencized', ignore_index=True).reset_index(drop=True)

In [424]:
df_jobs['Job Description Job Description spacy_sentencized_lower'] = df_jobs['Job Description spacy_sentencized'].apply(
    lambda job_sentence: job_sentence.strip().lower()
)


In [425]:
len(df_jobs)

135

In [444]:
df_jobs['Job Description spacy_tokenized'] = df_jobs['Job Description spacy_sentencized'].apply(
    lambda job_sentence: [
        str(token.text.strip().lower()) 
        for token in nlp.tokenizer(job_sentence) 
        if len(token) != 0 
        and not token.is_stop 
        and not token.is_punct 
        and not token.text in custom_punct_chars
    ]
)

In [445]:
df_jobs['Job Description spacy_tokenized']

0                                              [purpose]
1                                     [studio, director]
2                                    [special, projects]
3      [adidas, digital, task, manage, deliver, studi...
4      [program, scope, comprise, multiple, countries...
                             ...                        
130    [strong, ms, office, skills, word, excel, powe...
131            [basic, experience, broad, understanding]
132    [working, knowledge, agile, working, methods, ...
133    [working, knowledge, pmi, methods, ideally, ce...
134                 [fluent, english, verbally, written]
Name: Job Description spacy_tokenized, Length: 135, dtype: object

In [426]:
# Load NLK
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer, SnowballStemmer, PorterStemmer, LancasterStemmer

nltk_path = f'{llm_path}/nltk'
nltk.data.path.append(nltk_path)

nltk.download('words', download_dir = nltk_path)
nltk.download('stopwords', download_dir = nltk_path)
nltk.download('punkt', download_dir = nltk_path)
nltk.download('averaged_perceptron_tagger', download_dir = nltk_path)

stop_words = set(stopwords.words('english'))
punctuations = list(string.punctuation)


[nltk_data] Downloading package words to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package averaged_perceptron_

In [279]:
# pattern = r'[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?<=[a-z])(?=[A-Z])'
pattern = r'[\n]+|[,]{2,}|[|]{2,}|[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?=\:+[A-Z])'

pattern_numbers = r'[^a-zA-Z0-9]'

In [280]:
# nltk_sentencized = []

# for job_description in job_descriptions:
#     for sentence in sent_tokenize(job_description):
#         for sent in re.split(pattern, sentence):
#             nltk_sentencized.append(sent)

In [281]:
nltk_sentencized = [
    sent 
    for job_description in job_descriptions 
    for sentence in sent_tokenize(job_description) 
    for sent in re.split(pattern, sentence)
    if len(sent) != 0
]

In [282]:
len(nltk_sentencized)

133

In [283]:
# df_jobs['Job Description nltk_sentencized'] = df_jobs['Job Description'].apply(
#     lambda job_description: [
#         sent 
#         for sentence in sent_tokenize(job_description) 
#         for sent in re.split(pattern, sentence)
#         if len(sent) != 0
#     ]
# )

In [284]:
# df_jobs['Job Description nltk_sentencized'] = df_jobs['Job Description'].apply(
#     lambda job_description: [
#         re.split(pattern, sentence)
#         for sentence in sent_tokenize(job_description)
#         if len(sentence) != 0
#     ]
# )

In [428]:
# df_jobs['Job Description nltk_sentencized'][0][-10:-1]

In [429]:
# len(df_jobs['Job Description nltk_sentencized'][0]) + len(df_jobs['Job Description nltk_sentencized'][1])


In [287]:
# df_jobs = df_jobs.explode('Job Description nltk_sentencized', ignore_index=True).reset_index(drop=True)

In [288]:
# len(df_jobs)

133

In [290]:
# df_jobs['Job Description nltk_sentencized']

0                                                Purpose
1                                 As a Studio A Director
2                                       Special Projects
3      in adidas Digital your task is to manage and d...
4      The program scope may comprise multiple countr...
                             ...                        
128    Strong MS-Office skills (Word, Excel, PowerPoint)
129     Basic Experience and a broad understanding of IT
130    Working knowledge of Agile working methods for...
131    Working knowledge of PMI methods ideally with ...
132             Fluent English both verbally and written
Name: Job Description nltk_sentencized, Length: 133, dtype: object

In [31]:
nltk_sentencized_lower = [
    str(sent.strip().lower()) 
    for sent in nltk_sentencized
]

In [33]:
len(nltk_sentencized_lower)

133

In [295]:
# df_jobs['Job Description nltk_sentencized_lower'] = df_jobs['Job Description nltk_sentencized'].apply(
#     lambda job_sentence: job_sentence.strip().lower()
# )


In [430]:
# df_jobs['Job Description nltk_sentencized_lower']


In [431]:
for index, sentence in df_jobs['Job Description spacy_sentencized'].items():
    if str_fix_eg in sentence:
        print(index)


18


In [433]:
df_jobs['Job Description spacy_sentencized'][17:25].to_list()

['Form, lead, manage and monitor multi-functional team resources (internals and externals) to deliver and support projects & programs.',
 'Partner with Procurement in order to manage suppliers for the projects & programs in scope, for example interviews, staffing, supplier costs.',
 'Communication   stakeholder management',
 'Act as the key contact person for strategic stakeholders.',
 'Apply appropriate and effective communication methods to senior management and important stakeholders (including vendor) throughout the project lifecycle.',
 'As conflicts and escalations arise within projects, identify solutions and manage the resolution in a timely and appropriate manner.',
 'Drive change management activities for respective projects & programs and ensure changes are smoothly and successfully implemented to achieve lasting benefits.',
 'Project Controlling']

In [434]:
for index, sentence in df_jobs['Job Description spacy_sentencized'].items():
    if 'Power' in sentence:
        print(index)

63
130


In [435]:
df_jobs['Job Description spacy_sentencized'][60:70].to_list()

['Highly developed leadership skills are required',
 'Hard-Skills',
 'Knowledge of digital technologies and communications platforms and culture',
 'Strong MS-Office skills (Word, Excel, PowerPoint)',
 'Ability to travel, domestic or international, as required',
 'In-depth understanding of IT',
 'Fluent English both verbally and written',
 'Purpose:',
 'As a Manager Digital Strategy and Programs in adidas market ecom organization your task is to manage and deliver small digital projects, significant RFCs or sub-elements of mid-scale digital projects.',
 'The project scope is on a local (market) level across brands and functions.']

In [208]:
for idx, sent in enumerate(nltk_sentencized):
    if str_fix_eg in sent:
        print(idx)

In [35]:
next(idx for idx, sent in enumerate(nltk_sentencized) if str_fix_eg in sent)

17

In [36]:
# str_fix = 'Apply appropriate and effective communication methods to senior management and important stakeholders \(incl.'

for idx, sent in enumerate(nltk_sentencized):
    if str_fix_incl.split('\(')[0] in sent:
        print(idx)

20
84


In [37]:
nltk_sentencized[84:95]

['Apply appropriate and effective communication methods to senior management and important stakeholders throughout the project lifecycle.',
 'As conflicts and escalations arise within projects, identify solutions and drive their resolution in a timely and appropriate manner.',
 'Manage change within projects and ensure changes are smoothly and successfully implemented to achieve lasting benefits.',
 'Project Controlling',
 'Manage project controlling as an independent element to ensure that actual project costs are in line the committed budget.',
 'Conduct engagement reviews.',
 'Verify compliance with quality assurance procedures.',
 'Validate if project are delivering against program KPIs.',
 'Monitor project variables (cost, effort, scope, et cetera) against plan in order to implement corrective or preventive actions.',
 '"If required" Responsibilities',
 'Agile Transformation']

In [38]:
# sentences split on e.g. and incl.
nltk_sentencized[
    next(idx for idx, sent in enumerate(nltk_sentencized) if str_fix_eg in sent):next(idx for idx, sent in enumerate(nltk_sentencized) if str_fix_incl.split('\(')[0] in sent)+2
]


['Partner with Procurement in order to manage suppliers for the projects & programs in scope, for example interviews, staffing, supplier costs.',
 'Communication   stakeholder management',
 'Act as the key contact person for strategic stakeholders.',
 'Apply appropriate and effective communication methods to senior management and important stakeholders (including vendor) throughout the project lifecycle.',
 'As conflicts and escalations arise within projects, identify solutions and manage the resolution in a timely and appropriate manner.']

In [39]:
for idx, sent in enumerate(nltk_sentencized):
    if 'Power' in sent:
        print(idx)

62
128


In [40]:
next(idx for idx, sent in enumerate(nltk_sentencized) if 'Power' in sent)

62

In [41]:
nltk_sentencized[next(idx for idx, sent in enumerate(nltk_sentencized) if 'Power' in sent)]

'Strong MS-Office skills (Word, Excel, PowerPoint)'

In [42]:
for idx, sentence in enumerate(nltk_sentencized[84:95]):
    print(f'Sentence {idx+1}: {sentence}\n')

Sentence 1: Apply appropriate and effective communication methods to senior management and important stakeholders throughout the project lifecycle.

Sentence 2: As conflicts and escalations arise within projects, identify solutions and drive their resolution in a timely and appropriate manner.

Sentence 3: Manage change within projects and ensure changes are smoothly and successfully implemented to achieve lasting benefits.

Sentence 4: Project Controlling

Sentence 5: Manage project controlling as an independent element to ensure that actual project costs are in line the committed budget.

Sentence 6: Conduct engagement reviews.

Sentence 7: Verify compliance with quality assurance procedures.

Sentence 8: Validate if project are delivering against program KPIs.

Sentence 9: Monitor project variables (cost, effort, scope, et cetera) against plan in order to implement corrective or preventive actions.

Sentence 10: "If required" Responsibilities

Sentence 11: Agile Transformation



In [43]:
# nltk_tokenized = []

# for job_sentence in nltk_sentencized:
#     for token in word_tokenize(job_sentence):
#         if len(token) != 0 and token != '...' and token.lower() not in set(stopwords.words('english')) and token.lower() not in list(string.punctuation):
#             nltk_tokenized.append(str(token.strip().lower())))


In [44]:
nltk_tokenized = [
    str(token.strip().lower()) 
    for job_sentence in nltk_sentencized 
    for token in word_tokenize(job_sentence) 
    if len(token) != 0 
    and token != '...' 
    and not token.lower() in set(stopwords.words('english')) 
    and not token.lower() in list(string.punctuation) 
]

In [436]:
df_jobs['Job Description nltk_tokenized'] = df_jobs['Job Description spacy_sentencized'].apply(
    lambda job_sentence: [
        str(token.strip().lower()) 
        for token in word_tokenize(job_sentence) 
        if len(token) != 0 
        and token != '...' 
        and not token.lower() in set(stopwords.words('english')) 
        and not token.lower() in list(string.punctuation) 
    ]
)

In [437]:
df_jobs['Job Description nltk_tokenized']

0                                              [purpose]
1                                     [studio, director]
2                                    [special, projects]
3      [adidas, digital, task, manage, deliver, studi...
4      [program, scope, may, comprise, multiple, coun...
                             ...                        
130    [strong, ms-office, skills, word, excel, power...
131            [basic, experience, broad, understanding]
132    [working, knowledge, agile, working, methods, ...
133    [working, knowledge, pmi, methods, ideally, ce...
134                 [fluent, english, verbally, written]
Name: Job Description nltk_tokenized, Length: 135, dtype: object

In [45]:
'...' in nltk_tokenized

False

In [46]:
len(nltk_tokenized)

915

In [49]:
nltk_token_tags[1][1]

'NN'

In [47]:
nltk_token_tags = pos_tag(nltk_tokenized)

In [511]:
df_jobs['Job Description nltk_token_tags'] = df_jobs['Job Description nltk_tokenized'].apply(
    lambda token: pos_tag(token)
)


In [512]:
df_jobs['Job Description nltk_token_tags']

0                                        [(purpose, NN)]
1                         [(studio, NN), (director, NN)]
2                       [(special, JJ), (projects, NNS)]
3      [(adidas, JJ), (digital, JJ), (task, NN), (man...
4      [(program, NN), (scope, NN), (may, MD), (compr...
                             ...                        
130    [(strong, JJ), (ms-office, NN), (skills, NNS),...
131    [(basic, JJ), (experience, NN), (broad, JJ), (...
132    [(working, VBG), (knowledge, NN), (agile, IN),...
133    [(working, VBG), (knowledge, NN), (pmi, NN), (...
134    [(fluent, JJ), (english, JJ), (verbally, RB), ...
Name: Job Description nltk_token_tags, Length: 135, dtype: object

In [50]:
# https://www.machinelearningplus.com/nlp/lemmatization-examples-python/

In [440]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([token])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [441]:
import unicodedata

lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')

nltk_lemmas = []
nltk_stems = []

for token in nltk_tokenized:
    token = unicodedata.normalize('NFKD', str(token.strip().lower())).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    token_lemma = lemmatizer.lemmatize(token)
    token_stem = stemmer.stem(token)
    token_pos = get_wordnet_pos(token)
    token_pos_lemma = lemmatizer.lemmatize(token, token_pos)
    print(f'Token: {token} | Stem: {token_stem} | Lemma: {token_lemma} | POS Lemma: {token_pos_lemma}')
    nltk_lemmas.append(token_pos_lemma)
    nltk_stems.append(token_stem)


Token: purpose | Stem: purpos | Lemma: purpose | POS Lemma: purpose
Token: studio | Stem: studio | Lemma: studio | POS Lemma: studio
Token: director | Stem: director | Lemma: director | POS Lemma: director
Token: special | Stem: special | Lemma: special | POS Lemma: special
Token: projects | Stem: project | Lemma: project | POS Lemma: project
Token: adidas | Stem: adida | Lemma: adidas | POS Lemma: adidas
Token: digital | Stem: digit | Lemma: digital | POS Lemma: digital
Token: task | Stem: task | Lemma: task | POS Lemma: task
Token: manage | Stem: manag | Lemma: manage | POS Lemma: manage
Token: deliver | Stem: deliv | Lemma: deliver | POS Lemma: deliver
Token: studio | Stem: studio | Lemma: studio | POS Lemma: studio
Token: programs | Stem: program | Lemma: program | POS Lemma: program
Token: including | Stem: includ | Lemma: including | POS Lemma: include
Token: full | Stem: full | Lemma: full | POS Lemma: full
Token: projects | Stem: project | Lemma: project | POS Lemma: project
To

In [ ]:
import unicodedata

lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')

nltk_lemmas = []
nltk_stems = []

for token in nltk_tokenized:
    token = unicodedata.normalize('NFKD', str(token.strip().lower())).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    token_lemma = lemmatizer.lemmatize(token)
    token_stem = stemmer.stem(token)
    token_pos = get_wordnet_pos(token)
    nltk_lemmas.append(lemmatizer.lemmatize(token, get_wordnet_pos(token)))
    nltk_stems.append(token_stem)


In [318]:
nltk_lemmas = [
    lemmatizer.lemmatize(
        token, get_wordnet_pos(
            unicodedata.normalize('NFKD', str(token.strip().lower())).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        )
    )
    for token in nltk_tokenized
]

In [454]:
df_jobs['Job Description nltk_lemmas'] = df_jobs['Job Description spacy_tokenized'].apply(
    lambda tokens: [
        lemmatizer.lemmatize(
            token, get_wordnet_pos(
                unicodedata.normalize('NFKD', str(token.strip().lower())).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            )
        )
        for token in tokens
    ]
)


In [455]:
df_jobs['Job Description nltk_lemmas']

0                                              [purpose]
1                                     [studio, director]
2                                     [special, project]
3      [adidas, digital, task, manage, deliver, studi...
4      [program, scope, comprise, multiple, countries...
                             ...                        
130    [strong, ms, office, skills, word, excel, powe...
131               [basic, experience, broad, understand]
132    [work, knowledge, agile, work, methods, exampl...
133    [work, knowledge, pmi, methods, ideally, certi...
134                   [fluent, english, verbally, write]
Name: Job Description nltk_lemmas, Length: 135, dtype: object

In [320]:
len(nltk_lemmas)

915

In [ ]:
import unicodedata

lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')

nltk_lemmas = []
nltk_stems = []

for token in nltk_tokenized:
    token = unicodedata.normalize('NFKD', str(token.strip().lower())).encode('ascii', 'ignore').decode('utf-8', 'ignore')

    nltk_stems.append(stemmer.stem(unicodedata.normalize('NFKD', str(token.strip().lower())).encode('ascii', 'ignore').decode('utf-8', 'ignore')))


In [321]:
nltk_stems = [
    stemmer.stem(
        unicodedata.normalize('NFKD', str(token.strip().lower())).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    )
    for token in nltk_tokenized
]

In [458]:
df_jobs['Job Description nltk_stems'] = df_jobs['Job Description spacy_tokenized'].apply(
    lambda tokens: [
        stemmer.stem(
            unicodedata.normalize('NFKD', str(token.strip().lower())).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        )
        for token in tokens
    ]
)

In [459]:
df_jobs['Job Description nltk_stems']

0                                               [purpos]
1                                     [studio, director]
2                                     [special, project]
3      [adida, digit, task, manag, deliv, studio, pro...
4      [program, scope, compris, multipl, countri, fu...
                             ...                        
130    [strong, ms, offic, skill, word, excel, powerp...
131                   [basic, experi, broad, understand]
132    [work, knowledg, agil, work, method, exampl, s...
133         [work, knowledg, pmi, method, ideal, certif]
134                   [fluent, english, verbal, written]
Name: Job Description nltk_stems, Length: 135, dtype: object

In [322]:
len(nltk_stems)

915

In [338]:
from gensim.utils import save_as_line_sentence, simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords, preprocess_string, preprocess_documents

gensim_tokenized = []

for job_sentence in nltk_sentencized:
    gensim_tokenized.append(preprocess_string(re.sub(pattern_numbers, ' ', job_sentence.strip().lower())))

In [446]:
gensim_tokenized[0]

['purpos']

In [442]:
df_jobs['Job Description gensim_tokenized'] = df_jobs['Job Description spacy_sentencized'].apply(
    lambda job_sentence: preprocess_string(re.sub(pattern_numbers, ' ', job_sentence.strip().lower()))
)


In [443]:
df_jobs['Job Description gensim_tokenized']

0                                               [purpos]
1                                     [studio, director]
2                                     [special, project]
3      [adida, digit, task, manag, deliv, studio, pro...
4      [program, scope, compris, multipl, countri, fu...
                             ...                        
130      [strong, offic, skill, word, excel, powerpoint]
131                   [basic, experi, broad, understand]
132    [work, knowledg, agil, work, method, exampl, s...
133         [work, knowledg, pmi, method, ideal, certif]
134                   [fluent, english, verbal, written]
Name: Job Description gensim_tokenized, Length: 135, dtype: object

In [349]:
# special_cases = {":)": [{"ORTH": ":)"}]}
# prefix_re = re.compile(r'''^[\\[\\("']''')
# suffix_re = re.compile(r'''[\\]\\)"']$''')
# infix_re = re.compile(r'''[-~]''')
# simple_url_re = re.compile(r'''^https?://''')

# def custom_tokenizer(nlp):
#     return Tokenizer(nlp.vocab, rules=special_cases,
#                                 prefix_search=prefix_re.search,
#                                 suffix_search=suffix_re.search,
#                                 infix_finditer=infix_re.finditer,
#                                 url_match=simple_url_re.match)

In [350]:
# nlp.tokenizer.add_special_case('incl', [{ORTH: u'incl', NORM: u'include', LEMMA: u'include', POS: u'VERB'}])


In [353]:
# # Spacy sentencizer
# spacy_sentencized = []

# if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
#     for job_description in job_descriptions:
#         for sentence in nlp(job_description).sents:
#             for sent in re.split(pattern, sentence.text):
#                 if len(sent) != 0:
#                     spacy_sentencized.append(sent)

In [65]:
# Spacy sentencizer
if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    spacy_sentencized = [
        sent 
        for job_description in job_descriptions 
        for sentence in nlp(job_description).sents 
        for sent in re.split(pattern, sentence.text) 
        if len(sent) != 0 
    ]
    
    spacy_sentencized_lower = [
        str(sent.strip().lower()) 
        for sent in spacy_sentencized
    ]


In [66]:
# # Spacy sentencizer
# if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
#     spacy_sentencized = [
#         re.split(pattern, sentence.text) 
#         for job_description in job_descriptions 
#         for sentence in nlp(job_description).sents 
# #         for sent in re.split(pattern, sentence.text) 
#         if len(sentence) != 0 
#     ]



In [67]:
spacy_sentencized[23]

'Drive change management activities for respective projects & programs and ensure changes are smoothly and successfully implemented to achieve lasting benefits.'

In [68]:
# # Spacy tokenizer
# spacy_tokenized = []

# if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
#     for job_sentence in spacy_sentencized:
#         for token in nlp.tokenizer(job_sentence):
#             if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars:
#                 spacy_tokenized.append(token.text)

In [69]:
type(spacy_sentencized[0])

str

In [70]:
len(spacy_sentencized)

135

In [71]:
for idx, sent in enumerate(spacy_sentencized):
    if str_fix_eg in sent:
        print(idx)

18


In [72]:
next(idx for idx, sent in enumerate(spacy_sentencized) if str_fix_eg in sent)

18

In [73]:
# str_fix = 'Apply appropriate and effective communication methods to senior management and important stakeholders \(incl.'


for idx, sent in enumerate(spacy_sentencized):
    if str_fix_incl.split('\(')[0] in sent:
        print(idx)

21
85


In [74]:
next(idx for idx, sent in enumerate(spacy_sentencized) if str_fix_incl.split('\(')[0] in sent)

21

In [75]:
spacy_sentencized[18:25]

['Partner with Procurement in order to manage suppliers for the projects & programs in scope, for example interviews, staffing, supplier costs.',
 'Communication   stakeholder management',
 'Act as the key contact person for strategic stakeholders.',
 'Apply appropriate and effective communication methods to senior management and important stakeholders (including vendor) throughout the project lifecycle.',
 'As conflicts and escalations arise within projects, identify solutions and manage the resolution in a timely and appropriate manner.',
 'Drive change management activities for respective projects & programs and ensure changes are smoothly and successfully implemented to achieve lasting benefits.',
 'Project Controlling']

In [76]:
for idx, sent in enumerate(spacy_sentencized):
    if 'Power' in sent:
        print(idx)

63
130


In [77]:
next(idx for idx, sent in enumerate(spacy_sentencized) if 'Power' in sent)

63

In [78]:
spacy_sentencized[63]

'Strong MS-Office skills (Word, Excel, PowerPoint)'

In [79]:
for idx, sentence in enumerate(spacy_sentencized[18:25]):
    print(f'Sentence {idx+1}: {sentence}\n')

Sentence 1: Partner with Procurement in order to manage suppliers for the projects & programs in scope, for example interviews, staffing, supplier costs.

Sentence 2: Communication   stakeholder management

Sentence 3: Act as the key contact person for strategic stakeholders.

Sentence 4: Apply appropriate and effective communication methods to senior management and important stakeholders (including vendor) throughout the project lifecycle.

Sentence 5: As conflicts and escalations arise within projects, identify solutions and manage the resolution in a timely and appropriate manner.

Sentence 6: Drive change management activities for respective projects & programs and ensure changes are smoothly and successfully implemented to achieve lasting benefits.

Sentence 7: Project Controlling



In [80]:
# # Spacy tokenizer
# spacy_tokenized = []

# for job_sentence in spacy_sentencized:
# #         doc = nlp.tokenizer(job_sentence)
#     spacy_tokenized.extend(
#         [
#             token.text for token in nlp.tokenizer(job_sentence) 
#             if token.text not in custom_punct_chars
#             and not token.is_stop 

#         ]
#     )

In [81]:
type(spacy_sentencized[0])

str

In [82]:
spacy_sentencized[6]

'Planning, executing and tracking projects & programs in cooperation with the respective functions and project team members are a few examples of what is expected from this role.'

In [83]:
# Spacy tokenizer

spacy_tokenized = [
    str(token.text.strip().lower()) 
    for job_sentence in spacy_sentencized 
    for token in nlp.tokenizer(job_sentence) 
    if len(token) != 0 
    and not token.is_stop 
    and not token.is_punct 
    and not token.text in custom_punct_chars
]


In [84]:
len(spacy_tokenized)

910

In [85]:
spacy_tokenized[:10]

['purpose',
 'studio',
 'director',
 'special',
 'projects',
 'adidas',
 'digital',
 'task',
 'manage',
 'deliver']

In [86]:
# nlp.get_pipe('attribute_ruler').add([[{"TEXT":"Angeltown"}]],{"LEMMA":"San Fransisco"})

In [87]:
# # Spacy pos tagging
# nlp_stemmer = LancasterStemmer()
# nlp_stemmer = PorterStemmer()

nlp_token_tags = []
nlp_lemmas = []
nlp_stems = []

for job_description in job_descriptions:
    for token in nlp(job_description):
        if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars:
            nlp_token_tags.append(tuple([token.text, token.tag_]))
            nlp_lemmas.append(token.lemma_)
            nlp_stems.append(stemmer.stem(token.text))
#         for sentence in doc.sents:
#             for sent in re.split(pattern, sentence.text):
#                 if len(sent) != 0:
#                     spacy_sentencized.append(sent)

In [483]:
nlp_token_tags = [
    tuple([token.text.strip().lower(), token.tag_])
    for job_description in job_descriptions
    for token in nlp(job_description)
    if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars
]


In [479]:
df_jobs['Job Description nlp_token_tags'] = df_jobs['Job Description spacy_sentencized'].apply(
    lambda job_sentence: [
        tuple([token.text.strip().lower(), token.tag_])
        for token in nlp(job_sentence)
        
    ]
)


In [481]:
nlp_lemmas = [
    token.lemma_.strip().lower()
    for job_description in job_descriptions
    for token in nlp(job_description)
    if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars
]

In [482]:
df_jobs['Job Description nlp_lemmas'] = df_jobs['Job Description spacy_sentencized'].apply(
    lambda job_sentence: [
        token.lemma_.strip().lower()
        for token in nlp(job_sentence)
        if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars
    ]
)


In [484]:
nlp_stems = [
    stemmer.stem(token.text.strip().lower())
    for job_description in job_descriptions
    for token in nlp(job_description)
    if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars
]

In [485]:
df_jobs['Job Description nlp_stems'] = df_jobs['Job Description spacy_sentencized'].apply(
    lambda job_sentence: [
        stemmer.stem(token.text.strip().lower())
        for token in nlp(job_sentence)
        if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars
    ]
)


In [486]:
df_jobs['Job Description nlp_stems']

0                                               [purpos]
1                                     [studio, director]
2                                     [special, project]
3      [adida, digit, task, manag, deliv, studio, pro...
4      [program, scope, compris, multipl, countri, fu...
                             ...                        
130    [strong, ms, offic, skill, word, excel, powerp...
131                   [basic, experi, broad, understand]
132    [work, knowledg, agil, work, method, exampl, s...
133         [work, knowledg, pmi, method, ideal, certif]
134                   [fluent, english, verbal, written]
Name: Job Description nlp_stems, Length: 135, dtype: object

In [88]:
# if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
#     nlp_token_tags = [
#         tuple([token.text, token.tag_]) 
#         for job_description in job_descriptions 
#         for token in nlp(job_description) 
#         if len(token) != 0 
#         and not token.is_stop 
#         and not token.is_punct 
#         and not token.text in custom_punct_chars
#     ]

In [89]:
def get_word_num_and_frequency(row, text_col):

    row['num_words'] = len(str(row[f'{text_col}']).split())
    row['num_unique_words'] = len(set(str(row[f'{text_col}']).split()))
    row['num_chars'] = len(str(row[f'{text_col}']))
    row['num_punctuations'] = len([c for c in str(row[f'{text_col}']) if c in string.punctuation])

    return row


In [513]:
df_jobs = df_jobs.apply(lambda row: get_word_num_and_frequency(row=row, text_col='Job Description spacy_sentencized'), axis='columns')

In [90]:
# for sent in spacy_sentencized:
#     num_words = len(sent.split())
#     num_unique_words = len(set(sent.split()))
#     num_chars = len(sent)
#     num_punctuations = len([c for c in sent if c in string.punctuation])

In [516]:
def get_sentiment(df_jobs_to_be_processed, text_col, algo='vader', sentiment_range=(-1,1)):

    ## calculate sentiment
    if algo == 'vader':
        df_jobs_to_be_processed['sentiment'] = df_jobs_to_be_processed[text_col].apply(lambda x: SentimentIntensityAnalyzer().polarity_scores(x)['compound'] if isinstance(x, str) else np.nan)
    elif algo == 'textblob':
        df_jobs_to_be_processed['sentiment'] = df_jobs_to_be_processed[text_col].apply(lambda x: TextBlob(x).sentiment.polarity)
    ## rescaled
    if sentiment_range != (-1,1):
        df_jobs_to_be_processed['sentiment'] = preprocessing.MinMaxScaler(feature_range=sentiment_range).fit_transform(df_jobs_to_be_processed[['sentiment']])
    # print(df_jobs_to_be_processed[['sentiment']].describe().T)

    return df_jobs_to_be_processed


In [517]:
df_jobs = get_sentiment(df_jobs, text_col='Job Description spacy_sentencized')


In [92]:
# Sentiment
from nltk.sentiment.vader import SentimentIntensityAnalyzer

vader = SentimentIntensityAnalyzer()

for sent in spacy_sentencized:
    print(vader.polarity_scores(sent)['compound'])


0.0
0.0
0.4019
0.0
0.0
0.0
0.4215
0.0
0.0
0.0
0.0
0.0
0.6369
0.0
0.0
0.5719
0.0
0.4019
0.0
0.0
0.0
0.5994
-0.2263
0.8807
0.0
0.5719
0.4588
0.34
0.3818
0.0
0.0
0.8225
0.6369
0.3818
0.0
0.4404
0.0
0.0
0.3818
0.4215
0.0
0.0
0.0
0.0
0.0
0.4215
0.0
0.0
0.0
0.0
0.0
0.296
0.0
0.0
0.4927
0.3182
0.3182
0.5859
0.0
0.4404
0.0
0.0
0.0
0.743
0.3182
0.0
0.0
0.0
0.2023
0.0
0.0
0.4215
0.0
0.0
0.0
0.0
0.0
0.8442
0.3818
0.2023
0.4588
0.0
0.4019
0.0
0.0
0.5994
-0.2263
0.8126
0.0
0.5719
0.4588
0.34
0.3612
0.0
0.0
0.0
0.0
-0.128
-0.128
0.0
0.8402
0.4404
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.1779
0.4215
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.296
0.25
0.0
0.4404
0.3182
0.0
0.4404
0.0
0.743
0.0
0.0
0.4215
0.0


In [93]:
# # Sentiment
# from textblob import TextBlob, Word

# for sent in spacy_sentencized:
#     print(TextBlob(sent).sentiment.polarity)


In [548]:
# Bert Tokenizer
from transformers import BertTokenizer, AutoTokenizer, BertForPreTraining

bert_model_name = 'bert-base-uncased'
bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name, strip_accents = True)
bert_model = BertModel.from_pretrained(bert_model_name)

df_jobs['bert_tokenized'] = df_jobs['Job Description spacy_sentencized'].apply(lambda sentence: bert_tokenizer.tokenize(str(sentence)))



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
df_jobs['bert_tokenized']

In [ ]:
bert_tokenizer.encode_plus(sentence

In [543]:
def huggingface_similarity(sentences, bert_model):

    embeddings = bert_model.encode(sentences, show_progress_bar=True)
    cos_sim = util.cos_sim(embeddings, embeddings)

    #Sort list by the highest cosine similarity score
    all_sentence_combinations = sorted([[cos_sim[i][j], i, j] for i in range(len(cos_sim)-1) for j in range(i+1, len(cos_sim))], key=lambda x: x[0], reverse=True)

    if args['print_enabled'] is True:
        print('Top-5 most similar pairs:')
        for score, i, j in all_sentence_combinations[:5]:
            print(f'{sentences[i]} \t {sentences[j]} \t {cos_sim[i][j]:.4f}')

    if args['save_enabled'] is True:
        bert_model.to_json_file('bert_config.json')

    return all_sentence_combinations, embeddings

In [546]:
for sentences in spacy_sentencized:
    print(bert_tokenizer.encode_plus(sentences))

{'input_ids': [101, 3800, 102], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}
{'input_ids': [101, 2004, 1037, 2996, 1037, 2472, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}
{'input_ids': [101, 2569, 3934, 102], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}
{'input_ids': [101, 1999, 27133, 8883, 3617, 2115, 4708, 2003, 2000, 6133, 1998, 8116, 2996, 1037, 3454, 2164, 2440, 3934, 2346, 2863, 2361, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
{'input_ids': [101, 1996, 2565, 9531, 2089, 15821, 3674, 3032, 1998, 4972, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
{'input_ids': [101, 2115, 7863, 2003, 2000, 6204, 1996, 3934, 2306, 1996, 7372, 1997, 2051, 1010, 3465, 1004, 3737, 2096, 7995, 2006, 10390,

In [536]:
from transformers import AutoTokenizer, BertModel
import torch

tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
model = BertModel.from_pretrained(bert_model_name)

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [537]:
last_hidden_states

tensor([[[-0.1144,  0.1937,  0.1250,  ..., -0.3827,  0.2107,  0.5407],
         [ 0.5308,  0.3207,  0.3665,  ..., -0.0036,  0.7579,  0.0388],
         [-0.4877,  0.8849,  0.4256,  ..., -0.6976,  0.4458,  0.1231],
         ...,
         [-0.7003, -0.1815,  0.3297,  ..., -0.4838,  0.0680,  0.8901],
         [-1.0355, -0.2567, -0.0317,  ...,  0.3197,  0.3999,  0.1795],
         [ 0.6080,  0.2610, -0.3131,  ...,  0.0311, -0.6283, -0.1994]]],
       grad_fn=<NativeLayerNormBackward0>)

In [540]:
cls(
    BertModel.from_pretrained(
        bert_model_name,
        output_attentions=True,
        output_hidden_states=True,
        output_additional_info=True,
    ),
    BertAligner.from_pretrained(bert_model_name),
)

NameError: name 'cls' is not defined

In [ ]:
# Spacy sentencizer
# spacy_sentencized = []

if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    for job_description in job_descriptions:
        for token in nlp(job_description):
            print(token.text, token.tag_)
#             for sent in re.split(pattern, sentence.text):
#                 if len(sent) != 0:
#                     spacy_sentencized.append(sent)

In [ ]:
from spacy.matcher import Matcher
nlp = spacy.load('en_core_web_sm')

matcher = Matcher(nlp.vocab)

bigram_rules = [
    ['VERB', 'ADJ', 'NOUN'],
    ['NOUN', 'VERB', 'ADV'],
    ['NOUN', 'ADP', 'NOUN'],
    # more rules here...
]

rules = [
    ['VERB', 'ADJ', 'NOUN'],
    ['NOUN', 'VERB', 'ADV'],
    ['NOUN', 'ADP', 'NOUN'],
    # more rules here...
]

trigram_patterns = [[{"POS": i} for i in j] for j in rules]


In [ ]:
trigram_patterns

In [ ]:
patterns = {
    'noun_verb': [{'POS': 'NOUN'}, {'POS': 'VERB'}],
    'verb_noun': [{'POS': 'VERB'}, {'POS': 'NOUN'}],
    'adj_noun': [{'POS': 'ADJ'}, {'POS': 'NOUN'}],
    'adj_propn': [{'POS': 'ADJ'}, {'POS': 'PROPN'}],
}

In [ ]:
for pattern_name, pattern in patterns.items():
    matcher.add(pattern_name, [pattern])


In [ ]:
for job_description in job_descriptions:
    doc = nlp(job_description)
    matches = matcher(doc)

In [ ]:
for match_id, start, end in matches:
    
    # Get string representation
    string_id = nlp.vocab.strings[match_id]

    # The matched span
    span = doc[start:end]
    
    print(repr(span.text))
    print(match_id, string_id, start, end)
    print()

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


In [ ]:
# for sentence in spacy_sentencized:
#     print(model.encode(sentence))



In [ ]:
from transformers.models.electra.modeling_tf_electra import TFElectraMainLayer

In [ ]:
from transformers import AutoTokenizer, AutoModelForPreTraining

tokenizer = AutoTokenizer.from_pretrained("pavanchhatpar/electra-base-sentence-splitter")

model = AutoModelForPreTraining.from_pretrained("pavanchhatpar/electra-base-sentence-splitter", from_tf=True)


In [ ]:
nlp = pipeline('sentence-splitter', model)